In [1]:
import numpy as np
import os, glob
import os.path as osp

import matplotlib.pyplot as plt

from PIL import Image

from scipy.stats import ttest_ind

from mmaction.datasets.pipelines import Compose
import mmcv

# ..........torch imports............
import torch
import torchvision

from torch.utils.data import IterableDataset, DataLoader
from torchvision import transforms

#.... Captum imports..................
from captum.attr import LayerGradientXActivation, LayerIntegratedGradients

from captum.concept import TCAV
from captum.concept import Concept

from captum.concept._utils.data_iterator import dataset_to_dataloader, CustomIterableDataset
from captum.concept._utils.common import concepts_to_str

In [16]:
from mmaction.apis import init_recognizer, inference_recognizer

config_file = 'configs/recognition/swin/swin_tiny_patch244_window877_kinetics400_1k.py'
config = mmcv.Config.fromfile(config_file)
device = 'cpu' # or 'cpu'
device = torch.device(device)

model = init_recognizer(config_file, device=device, checkpoint='checkpoints/swin_tiny_patch244_window877_kinetics400_1k.pth')

Use load_from_local loader


In [17]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook
    
model.backbone.register_forward_hook(get_activation('cls_head.avg_pool'))

with torch.no_grad():
    output = model(input_tensors)
print(activation['backbone'])


12 12


KeyError: 'backbone'

In [15]:
activation['cls_head.avg_pool'].shape

torch.Size([4, 768, 16, 7, 7])

In [29]:
x = activation['backbone']
y = torch.mean(x, axis=0).reshape(-1, )

In [30]:
y.shape

torch.Size([768, 16, 7, 7])

In [4]:
# Method to normalize a video to Kinetics-400 mean and standard deviation
def transform(video, cfg=config):
    # build the data pipeline
    test_pipeline = cfg.data.test.pipeline
    data = dict(filename=video, label=-1, start_index=0, modality='RGB')
    if 'Init' not in test_pipeline[0]['type']:
        test_pipeline = [dict(type='OpenCVInit')] + test_pipeline
    else:
        test_pipeline[0] = dict(type='OpenCVInit')
    for i in range(len(test_pipeline)):
        if 'Decode' in test_pipeline[i]['type']:
            test_pipeline[i] = dict(type='OpenCVDecode')
    test_pipeline = Compose(test_pipeline)
    data = test_pipeline(data)
    data = data['imgs'].to(device)
    return data

In [5]:
def assemble_concept(name, id, concepts_path="../../shashank/kinetics-dataset/k400/avinab/interest_classes/"):
    concept_path = os.path.join(concepts_path, name) + "/"
    dataset = CustomIterableDataset(transform, concept_path)
    concept_iter = dataset_to_dataloader(dataset)

    return Concept(id=id, name=name, data_iter=concept_iter)

In [6]:
concepts_path = "../../shashank/kinetics-dataset/k400/avinab/interest_classes/"

sample1_concept = assemble_concept("sample1", 0, concepts_path=concepts_path)
sample2_concept = assemble_concept("sample2", 2, concepts_path=concepts_path)

In [18]:
layers=['backbone']

mytcav = TCAV(model=model,
              layers=layers)

/home/shashank/.conda/envs/video-swin/lib/python3.7/site-packages/captum/concept/_utils/classifier.py:131: UserWarning: Using default classifier for TCAV which keeps input both train and test datasets in the memory. Consider defining your own classifier that doesn't rely heavily on memory, for large number of concepts, by extending `Classifer` abstract class
  "Using default classifier for TCAV which keeps input"


In [19]:
experimental_set_rand = [[sample1_concept, sample2_concept]]
experimental_set_rand

[[Concept(0, 'sample1'), Concept(2, 'sample2')]]

In [20]:
# Load sample images from folder
input_tensors = torch.stack([transform(video) for video in glob.glob("../../shashank/kinetics-dataset/k400/avinab/interest_classes/sample3/*.mp4")])
#input_tensors = torch.randn([1, 24, 3, 224, 224])

In [21]:
input_tensors.shape

torch.Size([1, 12, 3, 32, 224, 224])

In [22]:
target_ind = 71

tcav_scores_w_random = mytcav.interpret(inputs=input_tensors,
                                        experimental_sets=experimental_set_rand,
                                        target=target_ind
                                       )
tcav_scores_w_random

12 12
12 12


/home/shashank/.conda/envs/video-swin/lib/python3.7/site-packages/captum/_utils/models/linear_model/train.py:351: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1666642814471/work/torch/csrc/utils/tensor_new.cpp:230.)
  bias_values = torch.FloatTensor([sklearn_model.intercept_]).to(  # type: ignore


12 12


defaultdict(<function captum.concept._core.tcav.TCAV.interpret.<locals>.<lambda>()>,
            {'0-2': defaultdict(None,
                         {'backbone': {'sign_count': tensor([1., 0.]),
                           'magnitude': tensor([ 5.9711e-06, -5.9711e-06])}})})